Veri Temizleme Senaryosu 2: Aykırı Değerler ve Tutarsız Metin Verisi
Bu senaryoda, bir şirketin çalışan performans değerlendirme verilerini incelediğimizi varsayalım. Veri setinde bazı çalışanların yaşları veya performans puanları gerçekçi olmayan değerlerde olabilir (aykırı değerler). Ayrıca, departman isimleri farklı şekillerde yazılmış olabilir (tutarsız metin verisi).

Şimdi bu durum için bir örnek veri seti oluşturalım:

In [5]:
import pandas as pd
import numpy as np

# Örnek Kirli Veri Seti Oluşturma (Senaryo 2 için)
data_employee = {
    'ÇalışanID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Adı': ['Ayşe', 'Burak', 'Cem', 'Deniz', 'Emre', 'Fatma', 'Gökhan', 'Hande', 'İrem', 'Jale'],
    'Yaş': [28, 35, 22, 45, 30, 29, 65, 27, 25, 120], # 65 ve 120 aykırı değer olabilir
    'Departman': ['IK', 'IT', 'Pazarlama', 'ik', 'BT', 'satış', 'IT', 'Pazarlama', 'IK', 'bt'], # Tutarsız departman isimleri
    'PerformansPuanı': [85, 92, 78, 95, 88, 80, 5, 90, 83, 91], # 5 aykırı değer olabilir
    'DeneyimYılı': [3, 7, 1, 10, 4, 3, 25, 2, 1, 6]
}

df_employee = pd.DataFrame(data_employee)
print("--- Ham Çalışan Veri Seti ---")
print(df_employee)
print("\n--- Veri Tipleri ---")
print(df_employee.info())

--- Ham Çalışan Veri Seti ---
   ÇalışanID     Adı  Yaş  Departman  PerformansPuanı  DeneyimYılı
0          1    Ayşe   28         IK               85            3
1          2   Burak   35         IT               92            7
2          3     Cem   22  Pazarlama               78            1
3          4   Deniz   45         ik               95           10
4          5    Emre   30         BT               88            4
5          6   Fatma   29      satış               80            3
6          7  Gökhan   65         IT                5           25
7          8   Hande   27  Pazarlama               90            2
8          9    İrem   25         IK               83            1
9         10    Jale  120         bt               91            6

--- Veri Tipleri ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ÇalışanID      

Yukarıdaki çıktıda Yaş ve PerformansPuanı sütunlarında potansiyel aykırı değerler ve Departman sütununda tutarsız metin verisi olduğunu görebiliriz.

Şimdi bu veri setini temizleyelim:

1. Aykırı Değerleri Tespit Etme ve Ele Alma (Yaş Sütunu):

İnsan yaşı için 65 ve 120 oldukça yüksek değerler. Genellikle 65 yaş ve üzeri çalışanlar azınlıkta olabilir veya 120 yaş, hatalı bir veri girişi olabilir. Birçok aykırı değer tespit yöntemi vardır (Z-skoru, IQR yöntemi). Basit bir yaklaşımla, Yaş için mantıksız bir üst sınır belirleyip bu değerleri düzeltebiliriz. Diyelim ki bir çalışanın yaşı 100'den büyük olamaz.

In [6]:
# Yaş sütunundaki aykırı değerleri tespit etme (100'den büyük olanlar)
outliers_age = df_employee[df_employee['Yaş'] > 100]
print("\n--- Yaş Kolonundaki Aykırı Değerler (100+) ---")
print(outliers_age)

# Aykırı yaş değerini mantıklı bir değerle değiştirme (örneğin ortalama veya NaN)
# Bu örnekte, 120 yaşındaki çalışanın yaşını ortalama ile değiştirelim.
# Genellikle veri analisti/mühendisi, domain bilgisi ile bu kararı verir.
mean_age = df_employee[df_employee['Yaş'] <= 100]['Yaş'].mean()
df_employee.loc[df_employee['Yaş'] > 100, 'Yaş'] = mean_age # mean_age ile değiştir
print("\n--- Yaş Kolonu Aykırı Değer Düzeltildikten Sonra ---")
print(df_employee)


--- Yaş Kolonundaki Aykırı Değerler (100+) ---
   ÇalışanID   Adı  Yaş Departman  PerformansPuanı  DeneyimYılı
9         10  Jale  120        bt               91            6

--- Yaş Kolonu Aykırı Değer Düzeltildikten Sonra ---
   ÇalışanID     Adı  Yaş  Departman  PerformansPuanı  DeneyimYılı
0          1    Ayşe   28         IK               85            3
1          2   Burak   35         IT               92            7
2          3     Cem   22  Pazarlama               78            1
3          4   Deniz   45         ik               95           10
4          5    Emre   30         BT               88            4
5          6   Fatma   29      satış               80            3
6          7  Gökhan   65         IT                5           25
7          8   Hande   27  Pazarlama               90            2
8          9    İrem   25         IK               83            1
9         10    Jale   34         bt               91            6


2. Yanlış Formatlı Verileri Düzeltme ve Veri Tipi Dönüşümü:

Fiyat (TL) sütununda '200 TL' gibi bir metin değeri var. Bunu sayısal bir değere dönüştürmemiz ve sütunun veri tipini float veya int yapmamız gerekiyor.


In [3]:
# ' TL' ifadesini kaldırıp sayıya çevirme
# Hata oluşursa (örneğin tamamen metin bir değer varsa) NaN yapar.
df['Fiyat (TL)'] = df['Fiyat (TL)'].astype(str).str.replace(' TL', '').astype(float)
print("\n--- Fiyat Kolonu Temizlendikten Sonra ---")
print(df)
print("\n--- Yeni Veri Tipleri ---")
print(df.info())


--- Fiyat Kolonu Temizlendikten Sonra ---
    SiparişID MüşteriAdı         Ürün  Fiyat (TL) SiparişTarihi  Adet
0         101        Ali       Laptop      2500.0    2023-01-15     1
1         102       Ayşe      Telefon      1200.0    18.02.2023     2
2         103       veli       Klavye       150.0    2023-03-10     1
3         104     Zeynep        Mouse        75.0    2023-04-05     3
4         105        Can      Monitör      1800.0    2023/05/20     1
5         106      Burak   Bilinmiyor       300.0    2023-06-01     1
6         103       veli       Klavye       150.0    2023-03-10     1
7         107      Deniz     Kulaklık       200.0    2023-07-22     2
8         108      Kadir  Ekran Kartı      1000.0    2023-07-05     1
9         109      Fatma         Mose       500.0    18.02.2023     3
10        115              Bilinmiyor       600.0    2023-01-15     2

--- Yeni Veri Tipleri ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 

3. Tekrarlayan Kayıtları Ele Alma:

SiparişID 103 olan ve MüşteriAdı 'veli' olan iki kayıt var. Bunlar büyük olasılıkla yinelenen girişler. Yinelenen satırları belirleyip silebiliriz. drop_duplicates() metoduyla bunu yaparız. subset parametresiyle hangi sütunlara bakarak yineleneni bulacağını belirtebiliriz. keep='first' ilkini koruyup diğerlerini siler.

In [4]:
# Tekrarlayan satırları kaldırma (SiparişID ve MüşteriAdı'na göre)
df.drop_duplicates(subset=['SiparişID', 'MüşteriAdı'], keep='first', inplace=True)
print("\n--- Tekrarlayan Kayıtlar Kaldırıldıktan Sonra ---")
print(df)


--- Tekrarlayan Kayıtlar Kaldırıldıktan Sonra ---
    SiparişID MüşteriAdı         Ürün  Fiyat (TL) SiparişTarihi  Adet
0         101        Ali       Laptop      2500.0    2023-01-15     1
1         102       Ayşe      Telefon      1200.0    18.02.2023     2
2         103       veli       Klavye       150.0    2023-03-10     1
3         104     Zeynep        Mouse        75.0    2023-04-05     3
4         105        Can      Monitör      1800.0    2023/05/20     1
5         106      Burak   Bilinmiyor       300.0    2023-06-01     1
7         107      Deniz     Kulaklık       200.0    2023-07-22     2
8         108      Kadir  Ekran Kartı      1000.0    2023-07-05     1
9         109      Fatma         Mose       500.0    18.02.2023     3
10        115              Bilinmiyor       600.0    2023-01-15     2


Yukarıdaki çıktıda Yaş ve PerformansPuanı sütunlarında potansiyel aykırı değerler ve Departman sütununda tutarsız metin verisi olduğunu görebiliriz.

Şimdi bu veri setini temizleyelim:

1. Aykırı Değerleri Tespit Etme ve Ele Alma (Yaş Sütunu):

İnsan yaşı için 65 ve 120 oldukça yüksek değerler. Genellikle 65 yaş ve üzeri çalışanlar azınlıkta olabilir veya 120 yaş, hatalı bir veri girişi olabilir. Birçok aykırı değer tespit yöntemi vardır (Z-skoru, IQR yöntemi). Basit bir yaklaşımla, Yaş için mantıksız bir üst sınır belirleyip bu değerleri düzeltebiliriz. Diyelim ki bir çalışanın yaşı 100'den büyük olamaz.

In [7]:
# Yaş sütunundaki aykırı değerleri tespit etme (100'den büyük olanlar)
outliers_age = df_employee[df_employee['Yaş'] > 100]
print("\n--- Yaş Kolonundaki Aykırı Değerler (100+) ---")
print(outliers_age)

# Aykırı yaş değerini mantıklı bir değerle değiştirme (örneğin ortalama veya NaN)
# Bu örnekte, 120 yaşındaki çalışanın yaşını ortalama ile değiştirelim.
# Genellikle veri analisti/mühendisi, domain bilgisi ile bu kararı verir.
mean_age = df_employee[df_employee['Yaş'] <= 100]['Yaş'].mean()
df_employee.loc[df_employee['Yaş'] > 100, 'Yaş'] = mean_age # mean_age ile değiştir
print("\n--- Yaş Kolonu Aykırı Değer Düzeltildikten Sonra ---")
print(df_employee)


--- Yaş Kolonundaki Aykırı Değerler (100+) ---
Empty DataFrame
Columns: [ÇalışanID, Adı, Yaş, Departman, PerformansPuanı, DeneyimYılı]
Index: []

--- Yaş Kolonu Aykırı Değer Düzeltildikten Sonra ---
   ÇalışanID     Adı  Yaş  Departman  PerformansPuanı  DeneyimYılı
0          1    Ayşe   28         IK               85            3
1          2   Burak   35         IT               92            7
2          3     Cem   22  Pazarlama               78            1
3          4   Deniz   45         ik               95           10
4          5    Emre   30         BT               88            4
5          6   Fatma   29      satış               80            3
6          7  Gökhan   65         IT                5           25
7          8   Hande   27  Pazarlama               90            2
8          9    İrem   25         IK               83            1
9         10    Jale   34         bt               91            6


2. Aykırı Değerleri Tespit Etme ve Ele Alma (PerformansPuanı Sütunu):

Performans puanları genellikle 0-100 arasında olur. PerformansPuanı 5 olan bir çalışanın puanı, diğerlerinden çok düşük ve aykırı bir değer olabilir. Bu bir veri girişi hatası veya olağanüstü bir durum olabilir.

In [8]:
# PerformansPuanı sütunundaki aykırı değerleri tespit etme (örneğin 10'dan küçük olanlar)
outliers_performance = df_employee[df_employee['PerformansPuanı'] < 10]
print("\n--- PerformansPuanı Kolonundaki Aykırı Değerler (10'dan küçük) ---")
print(outliers_performance)

# Aykırı Performans Puanını ortalama ile değiştirme veya NaN yapma
# Burada da ortalama ile değiştirelim (gerçek senaryoda domain uzmanı ile konuşmak gerekir)
mean_performance = df_employee[df_employee['PerformansPuanı'] >= 10]['PerformansPuanı'].mean()
df_employee.loc[df_employee['PerformansPuanı'] < 10, 'PerformansPuanı'] = mean_performance
print("\n--- PerformansPuanı Aykırı Değer Düzeltildikten Sonra ---")
print(df_employee)


--- PerformansPuanı Kolonundaki Aykırı Değerler (10'dan küçük) ---
   ÇalışanID     Adı  Yaş Departman  PerformansPuanı  DeneyimYılı
6          7  Gökhan   65        IT                5           25

--- PerformansPuanı Aykırı Değer Düzeltildikten Sonra ---
   ÇalışanID     Adı  Yaş  Departman  PerformansPuanı  DeneyimYılı
0          1    Ayşe   28         IK        85.000000            3
1          2   Burak   35         IT        92.000000            7
2          3     Cem   22  Pazarlama        78.000000            1
3          4   Deniz   45         ik        95.000000           10
4          5    Emre   30         BT        88.000000            4
5          6   Fatma   29      satış        80.000000            3
6          7  Gökhan   65         IT        86.888889           25
7          8   Hande   27  Pazarlama        90.000000            2
8          9    İrem   25         IK        83.000000            1
9         10    Jale   34         bt        91.000000            6


3. Tutarsız Metin Verilerini Standardize Etme (Departman Sütunu):

Departman sütununda 'IK', 'ik', 'IT', 'BT', 'Pazarlama', 'satış' gibi farklı yazılışlar var. Bunları standart hale getirmemiz gerekir. Örneğin, tümünü büyük harfe çevirip benzer anlamdaki kelimeleri birleştirebiliriz (IT ve BT).

In [9]:
# Departman isimlerini büyük harfe çevirme
df_employee['Departman'] = df_employee['Departman'].str.upper()

# Benzer anlamdaki departmanları birleştirme
df_employee['Departman'] = df_employee['Departman'].replace({'BT': 'IT', 'SATIŞ': 'PAZARLAMA'})
print("\n--- Departman Kolonu Standardize Edildikten Sonra ---")
print(df_employee)

# Standardize edilmiş departmanların benzersiz değerlerini kontrol edelim
print("\n--- Standardize Edilmiş Benzersiz Departmanlar ---")
print(df_employee['Departman'].unique())


--- Departman Kolonu Standardize Edildikten Sonra ---
   ÇalışanID     Adı  Yaş  Departman  PerformansPuanı  DeneyimYılı
0          1    Ayşe   28         IK        85.000000            3
1          2   Burak   35         IT        92.000000            7
2          3     Cem   22  PAZARLAMA        78.000000            1
3          4   Deniz   45         IK        95.000000           10
4          5    Emre   30         IT        88.000000            4
5          6   Fatma   29  PAZARLAMA        80.000000            3
6          7  Gökhan   65         IT        86.888889           25
7          8   Hande   27  PAZARLAMA        90.000000            2
8          9    İrem   25         IK        83.000000            1
9         10    Jale   34         IT        91.000000            6

--- Standardize Edilmiş Benzersiz Departmanlar ---
['IK' 'IT' 'PAZARLAMA']


Bu senaryoda, bir veri mühendisi olarak aykırı değerleri nasıl tespit edip yönetebileceğini ve tutarsız kategorik (metin) veriyi nasıl standardize edeceğini gördük. Her iki durumda da, veri setinin bağlamı (domain bilgisi) ve hedefin, nasıl bir temizleme stratejisi izleyeceğini belirlemede çok önemli olduğunu unutmamak gerekir.